# Build AI Works Database + Simplicial Complex (Keywords)

This notebook builds a filtered database of AI-related works from the OpenAlex processed data and provides a starting point for simplicial-complex analysis based on **keywords** (not topics).

**Key points**
- AI filtering is **not only primary topic**.
- Supports **AI strict** (primary topic is AI) and **AI broad** (AI appears in top-3 topics).
- Papers must satisfy:
  - year >= 1950
  - has DOI
  - has at least 1 reference **or** 1 citation
  - non-empty title
  - type in allowed list
  - language = `en`

Outputs: a parquet table with the needed columns.


In [3]:
import os
os.chdir("../") # resets notebook directory to repository root folder (DO ONLY ONCE!)
import random
import polars as pl
from pathlib import Path
from datetime import datetime
from tqdm.auto import tqdm


## 1) Configuration
Fill the AI topic IDs list (use your internal list or derive from topics table).

In [4]:
import pandas as pd

# Paths
ROOT = Path(".").resolve()
DATA = ROOT / "data"
WORKS_PARQUET_DIR = DATA / "works_by_topic_parquet"
REFERENCES_PARQUET_DIR = DATA / "works2references_by_topic_parquet"
CITATIONS_PARQUET_DIR = DATA / "works2citations_by_topic_parquet"
OUTPUT_DIR = DATA / "ai_database"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Filters
MIN_YEAR = 1950
ALLOWED_TYPES = {"article", "book-chapter", "review", "report", "book", "dissertation", "dataset", "preprint"}
LANGUAGE = "en"

# # AI Topics list (fill with IDs like "T12345")
# AI_TOPIC_IDS = set([
#     # TODO: populate with AI topic IDs
# ])
topics_df = pd.read_parquet("data/topics/topics.parquet")
AI_topics_df = topics_df[topics_df.subfield_name == "Artificial Intelligence"]
AI_TOPIC_IDS = set(list(AI_topics_df.topic_id))

# For testing: set to a small number of topic files to process
TEST_MODE = False
TEST_MAX_TOPICS = 3
RANDOM_SEED = 7


## Get topics

In [5]:
pl.Config.set_tbl_rows(100)
pd.set_option("display.max_columns", 100)

In [6]:
pl.Config.set_tbl_cols(100)
pd.set_option("display.max_rows", 100)

In [5]:
AI_topics_df.sort_values("works_count", ascending = False)

,topic_id,display_name,description,keywords,wikipedia,subfield_id,subfield_name,field_id,field_name,domain_id,domain_name,sibling_ids,works_count,cited_by_count
1,T12157,Geochemistry and Geologic Mapping,This cluster of papers focuses on the applicat...,Machine Learning_Mineral Prospectivity_Remote ...,https://en.wikipedia.org/wiki/Mineral_prospecting,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,3952624,1640349
37,T13650,Computational Physics and Python Applications,This cluster of papers covers a wide range of ...,Python_Scientific Computing_Statistical Modeli...,https://en.wikipedia.org/wiki/Python_(programm...,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T14175_T12611_T101...,409306,622443
88,T10181,Natural Language Processing Techniques,This cluster of papers focuses on statistical ...,Statistical Machine Translation_Neural Machine...,https://en.wikipedia.org/wiki/Statistical_mach...,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,283826,2999722
121,T10320,Neural Networks and Applications,This cluster of papers covers a wide range of ...,Neural Networks_Self-Organizing Maps_Backpropa...,https://en.wikipedia.org/wiki/Artificial_neura...,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,247207,4351608
151,T13398,Data Analysis with R,This cluster of papers focuses on statistical ...,R language_statistical analysis_data visualiza...,https://en.wikipedia.org/wiki/R_(programming_l...,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,224954,568812
250,T13734,Advanced Computational Techniques and Applicat...,This cluster of papers covers a wide range of ...,Expert Systems_Wavelet Analysis_Machine Learni...,https://en.wikipedia.org/wiki/Artificial_intel...,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,179457,201783
278,T10215,Semantic Web and Ontologies,This cluster of papers focuses on the Semantic...,Semantic Web_Ontology_Linked Data_RDF_OWL_Sche...,https://en.wikipedia.org/wiki/Semantic_Web,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,169782,1478650
308,T14335,Educational Robotics and Engineering,This cluster of papers focuses on robotics pro...,Robotics Programming_Education_Learning Enviro...,https://en.wikipedia.org/wiki/Educational_robo...,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,162423,24495
347,T13623,Linguistic Studies and Language Acquisition,This cluster of papers focuses on the compilat...,Spoken Language_Corpora_Italian_Portuguese_Pro...,https://en.wikipedia.org/wiki/Spoken_corpus,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,153071,208110
367,T10028,Topic Modeling,This cluster of papers covers a wide range of ...,Neural Networks_Word Representation_Machine Tr...,https://en.wikipedia.org/wiki/Natural_language...,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,150391,2301177


In [6]:
AI_topics_df["keywords_list"] = AI_topics_df.keywords.apply(lambda x : x.split("_"))
AI_topics_df

/tmp/6945126.1.all.q/ipykernel_4049248/684101191.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AI_topics_df["keywords_list"] = AI_topics_df.keywords.apply(lambda x : x.split("_"))


,topic_id,display_name,description,keywords,wikipedia,subfield_id,subfield_name,field_id,field_name,domain_id,domain_name,sibling_ids,works_count,cited_by_count,keywords_list
1,T12157,Geochemistry and Geologic Mapping,This cluster of papers focuses on the applicat...,Machine Learning_Mineral Prospectivity_Remote ...,https://en.wikipedia.org/wiki/Mineral_prospecting,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,3952624,1640349,"[Machine Learning, Mineral Prospectivity, Remo..."
37,T13650,Computational Physics and Python Applications,This cluster of papers covers a wide range of ...,Python_Scientific Computing_Statistical Modeli...,https://en.wikipedia.org/wiki/Python_(programm...,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T14175_T12611_T101...,409306,622443,"[Python, Scientific Computing, Statistical Mod..."
88,T10181,Natural Language Processing Techniques,This cluster of papers focuses on statistical ...,Statistical Machine Translation_Neural Machine...,https://en.wikipedia.org/wiki/Statistical_mach...,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,283826,2999722,"[Statistical Machine Translation, Neural Machi..."
121,T10320,Neural Networks and Applications,This cluster of papers covers a wide range of ...,Neural Networks_Self-Organizing Maps_Backpropa...,https://en.wikipedia.org/wiki/Artificial_neura...,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,247207,4351608,"[Neural Networks, Self-Organizing Maps, Backpr..."
151,T13398,Data Analysis with R,This cluster of papers focuses on statistical ...,R language_statistical analysis_data visualiza...,https://en.wikipedia.org/wiki/R_(programming_l...,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,224954,568812,"[R language, statistical analysis, data visual..."
250,T13734,Advanced Computational Techniques and Applicat...,This cluster of papers covers a wide range of ...,Expert Systems_Wavelet Analysis_Machine Learni...,https://en.wikipedia.org/wiki/Artificial_intel...,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,179457,201783,"[Expert Systems, Wavelet Analysis, Machine Lea..."
278,T10215,Semantic Web and Ontologies,This cluster of papers focuses on the Semantic...,Semantic Web_Ontology_Linked Data_RDF_OWL_Sche...,https://en.wikipedia.org/wiki/Semantic_Web,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,169782,1478650,"[Semantic Web, Ontology, Linked Data, RDF, OWL..."
308,T14335,Educational Robotics and Engineering,This cluster of papers focuses on robotics pro...,Robotics Programming_Education_Learning Enviro...,https://en.wikipedia.org/wiki/Educational_robo...,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,162423,24495,"[Robotics Programming, Education, Learning Env..."
347,T13623,Linguistic Studies and Language Acquisition,This cluster of papers focuses on the compilat...,Spoken Language_Corpora_Italian_Portuguese_Pro...,https://en.wikipedia.org/wiki/Spoken_corpus,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,153071,208110,"[Spoken Language, Corpora, Italian, Portuguese..."
367,T10028,Topic Modeling,This cluster of papers covers a wide range of ...,Neural Networks_Word Representation_Machine Tr...,https://en.wikipedia.org/wiki/Natural_language...,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences,T13674_T12380_T10951_T11010_T13650_T14175_T126...,150391,2301177,"[Neural Networks, Word Representation, Machine..."


In [22]:
out_AI_topics_df = AI_topics_df[["domain_id", "domain_name", "field_id", "field_name", "subfield_id", "subfield_name", "topic_id", "display_name", "wikipedia", "description", "keywords_list", "works_count", "cited_by_count"]].reset_index(drop=True)
out_AI_topics_df

,domain_id,domain_name,field_id,field_name,subfield_id,subfield_name,topic_id,display_name,wikipedia,description,keywords_list,works_count,cited_by_count
0,3,Physical Sciences,17,Computer Science,1702,Artificial Intelligence,T12157,Geochemistry and Geologic Mapping,https://en.wikipedia.org/wiki/Mineral_prospecting,This cluster of papers focuses on the applicat...,"[Machine Learning, Mineral Prospectivity, Remo...",3952624,1640349
1,3,Physical Sciences,17,Computer Science,1702,Artificial Intelligence,T13650,Computational Physics and Python Applications,https://en.wikipedia.org/wiki/Python_(programm...,This cluster of papers covers a wide range of ...,"[Python, Scientific Computing, Statistical Mod...",409306,622443
2,3,Physical Sciences,17,Computer Science,1702,Artificial Intelligence,T10181,Natural Language Processing Techniques,https://en.wikipedia.org/wiki/Statistical_mach...,This cluster of papers focuses on statistical ...,"[Statistical Machine Translation, Neural Machi...",283826,2999722
3,3,Physical Sciences,17,Computer Science,1702,Artificial Intelligence,T10320,Neural Networks and Applications,https://en.wikipedia.org/wiki/Artificial_neura...,This cluster of papers covers a wide range of ...,"[Neural Networks, Self-Organizing Maps, Backpr...",247207,4351608
4,3,Physical Sciences,17,Computer Science,1702,Artificial Intelligence,T13398,Data Analysis with R,https://en.wikipedia.org/wiki/R_(programming_l...,This cluster of papers focuses on statistical ...,"[R language, statistical analysis, data visual...",224954,568812
5,3,Physical Sciences,17,Computer Science,1702,Artificial Intelligence,T13734,Advanced Computational Techniques and Applicat...,https://en.wikipedia.org/wiki/Artificial_intel...,This cluster of papers covers a wide range of ...,"[Expert Systems, Wavelet Analysis, Machine Lea...",179457,201783
6,3,Physical Sciences,17,Computer Science,1702,Artificial Intelligence,T10215,Semantic Web and Ontologies,https://en.wikipedia.org/wiki/Semantic_Web,This cluster of papers focuses on the Semantic...,"[Semantic Web, Ontology, Linked Data, RDF, OWL...",169782,1478650
7,3,Physical Sciences,17,Computer Science,1702,Artificial Intelligence,T14335,Educational Robotics and Engineering,https://en.wikipedia.org/wiki/Educational_robo...,This cluster of papers focuses on robotics pro...,"[Robotics Programming, Education, Learning Env...",162423,24495
8,3,Physical Sciences,17,Computer Science,1702,Artificial Intelligence,T13623,Linguistic Studies and Language Acquisition,https://en.wikipedia.org/wiki/Spoken_corpus,This cluster of papers focuses on the compilat...,"[Spoken Language, Corpora, Italian, Portuguese...",153071,208110
9,3,Physical Sciences,17,Computer Science,1702,Artificial Intelligence,T10028,Topic Modeling,https://en.wikipedia.org/wiki/Natural_language...,This cluster of papers covers a wide range of ...,"[Neural Networks, Word Representation, Machine...",150391,2301177


In [23]:
out_AI_topics_df.to_csv(OUTPUT_DIR / "ai_topics.csv", index=False)

In [28]:
out_AI_topics_keywords_df = out_AI_topics_df[["display_name", "keywords_list"]].explode("keywords_list").rename(columns={"display_name":"topic_name", "keywords_list" : "keyword"})
out_AI_topics_keywords_df

,topic_name,keyword
0,Geochemistry and Geologic Mapping,Machine Learning
0,Geochemistry and Geologic Mapping,Mineral Prospectivity
0,Geochemistry and Geologic Mapping,Remote Sensing
0,Geochemistry and Geologic Mapping,Compositional Data Analysis
0,Geochemistry and Geologic Mapping,Geological Mapping
...,...,...
76,Experience-Based Knowledge Management,Internet of Things
76,Experience-Based Knowledge Management,Cognitive Vision Systems
76,Experience-Based Knowledge Management,Information Technology
76,Experience-Based Knowledge Management,Semantic-based Academic Social Network


In [29]:
out_AI_topics_keywords_df.to_csv(OUTPUT_DIR / "ai_topic_keywords_OA.csv", index=False)

## Some checks

In [6]:
import polars as pl
from pathlib import Path

# Point to a single topic file
topic_id = "T10969"
path = Path("data/works_by_topic_parquet") / f"{topic_id}.parquet"

df = pl.read_parquet(path, columns=["type"])
counts = (
    df.group_by("type")
      .agg(pl.len().alias("count"))
      .sort("count", descending=True)
)

print(counts)


shape: (17, 2)
┌─────────────────┬───────┐
│ type            ┆ count │
│ ---             ┆ ---   │
│ str             ┆ u32   │
╞═════════════════╪═══════╡
│ article         ┆ 76763 │
│ book-chapter    ┆ 7346  │
│ preprint        ┆ 5084  │
│ dissertation    ┆ 3885  │
│ book            ┆ 2475  │
│ dataset         ┆ 2154  │
│ other           ┆ 1327  │
│ review          ┆ 730   │
│ report          ┆ 612   │
│ peer-review     ┆ 407   │
│ libguides       ┆ 222   │
│ erratum         ┆ 85    │
│ editorial       ┆ 80    │
│ paratext        ┆ 78    │
│ reference-entry ┆ 72    │
│ letter          ┆ 50    │
│ standard        ┆ 1     │
└─────────────────┴───────┘


In [5]:
import polars as pl
from pathlib import Path

works_dir = Path("data/works_by_topic_parquet")
files = list(works_dir.glob("*.parquet"))[:10]  # change slice as needed

all_counts = []
for f in files:
    df = pl.read_parquet(f, columns=["type"])
    all_counts.append(df.group_by("type").agg(pl.len().alias("count")))

total = pl.concat(all_counts).group_by("type").agg(pl.sum("count").alias("count")).sort("count", descending=True)
print(total)


shape: (21, 2)
┌──────────────────┬────────┐
│ type             ┆ count  │
│ ---              ┆ ---    │
│ str              ┆ u32    │
╞══════════════════╪════════╡
│ article          ┆ 626212 │
│ dataset          ┆ 138625 │
│ other            ┆ 122654 │
│ book-chapter     ┆ 34203  │
│ dissertation     ┆ 24721  │
│ preprint         ┆ 22328  │
│ book             ┆ 10227  │
│ report           ┆ 2329   │
│ libguides        ┆ 1901   │
│ peer-review      ┆ 596    │
│ review           ┆ 429    │
│ reference-entry  ┆ 311    │
│ standard         ┆ 285    │
│ letter           ┆ 228    │
│ editorial        ┆ 109    │
│ paratext         ┆ 76     │
│ erratum          ┆ 12     │
│ book-section     ┆ 11     │
│ software         ┆ 9      │
│ database         ┆ 5      │
│ report-component ┆ 4      │
└──────────────────┴────────┘


In [6]:
import polars as pl
from pathlib import Path

works_dir = Path("data/works_by_topic_parquet")
files = list(works_dir.glob("*.parquet"))[:100]  # change slice as needed

all_counts = []
for f in files:
    df = pl.read_parquet(f, columns=["type"])
    all_counts.append(df.group_by("type").agg(pl.len().alias("count")))

total = (
    pl.concat(all_counts)
      .group_by("type")
      .agg(pl.sum("count").alias("count"))
      .with_columns((pl.col("count") / pl.col("count").sum() * 100).alias("pct"))
      .sort("count", descending=True)
)

print(total)


shape: (21, 3)
┌──────────────────┬─────────┬───────────┐
│ type             ┆ count   ┆ pct       │
│ ---              ┆ ---     ┆ ---       │
│ str              ┆ u32     ┆ f64       │
╞══════════════════╪═════════╪═══════════╡
│ article          ┆ 4584959 ┆ 64.11056  │
│ dataset          ┆ 1124034 ┆ 15.717141 │
│ book-chapter     ┆ 462975  ┆ 6.473686  │
│ other            ┆ 421753  ┆ 5.897287  │
│ dissertation     ┆ 188498  ┆ 2.63573   │
│ preprint         ┆ 166501  ┆ 2.32815   │
│ book             ┆ 128327  ┆ 1.794371  │
│ report           ┆ 22541   ┆ 0.315186  │
│ libguides        ┆ 21560   ┆ 0.301469  │
│ reference-entry  ┆ 10171   ┆ 0.142219  │
│ peer-review      ┆ 6761    ┆ 0.094538  │
│ review           ┆ 4126    ┆ 0.057693  │
│ standard         ┆ 3200    ┆ 0.044745  │
│ letter           ┆ 2352    ┆ 0.032888  │
│ report-component ┆ 1144    ┆ 0.015996  │
│ paratext         ┆ 1119    ┆ 0.015647  │
│ editorial        ┆ 1100    ┆ 0.015381  │
│ book-section     ┆ 245     ┆ 0.003426

## Get top papers in AI by citation count

In [33]:
base_folder = "data/works_by_topic_parquet2/"
columns = ["id", "date", "type", "language", "doi", "topics", "keywords", "cited_by_count_computed"]

dfs = []

for topic in tqdm(AI_TOPIC_IDS):
    path = os.path.join(base_folder, f"{topic}.parquet")

    df = pd.read_parquet(path, columns=columns)

    # # keep only columns that exist (prevents KeyError)
    # cols_present = [c for c in columns if c in df.columns]
    # df = df[cols_present].copy()

    # optional: keep track of which topic file it came from
    df["primary_topic"] = topic

    dfs.append(df)

works_df = pd.concat(dfs, ignore_index=True)

  0%|          | 0/77 [00:00<?, ?it/s]

In [36]:
works_df["date"]

0          2050-01-01
1          2030-01-01
2          2027-01-01
3          2026-05-01
4          2026-02-01
              ...    
8404572    1543-01-01
8404573    1541-01-01
8404574    1540-01-01
8404575    1517-01-01
8404576    1273-01-01
Name: date, Length: 8404577, dtype: string

In [37]:
works_df["date_str"] = works_df["date"].astype("string")

# decade bucket: first 3 chars, e.g. "195" for 1950s
works_df["decade3"] = works_df["date_str"].str.slice(0, 3)

# make cited count numeric + safe
works_df["cited_by_count_computed"] = pd.to_numeric(
    works_df["cited_by_count_computed"], errors="coerce"
).fillna(0).astype("int64")

# drop rows where decade3 isn't valid (e.g., missing/short date)
works_df_valid = works_df[works_df["decade3"].str.fullmatch(r"\d{3}", na=False)].copy()

# top 10 per decade (ties broken by date, then id)
top10_by_decade = (
    works_df_valid
    .sort_values(["decade3", "cited_by_count_computed", "date_str", "id"],
                 ascending=[True, False, True, True])
    .groupby("decade3", as_index=False)
    .head(10)
)

# optional: for nicer display, order decades ascending and ranks within each decade
top10_by_decade = top10_by_decade.sort_values(
    ["decade3", "cited_by_count_computed"], ascending=[True, False]
)

top10_by_decade

,id,date,type,language,doi,topics,keywords,cited_by_count_computed,primary_topic,decade3,date_str
2635490,W6990902826,1000-01-01,article,de,,T13559_0.1928;T11734_0.1642;T14216_0.1384,Set (abstract data type)_0.2836;Feature (lingu...,1,T13559,100,1000-01-01
157041,W7002581990,1000-10-10,other,en,,T13398_0.0623;T11986_0.0497;T13650_0.0359,Set (abstract data type)_0.2447;Identification...,0,T13398,100,1000-10-10
3550738,W7067165874,1061-01-01,article,en,,T13650_0.0343;T11986_0.0189;T12391_0.017,Government (linguistics)_0.2311;Natural (archa...,0,T13650,106,1061-01-01
716242,W7021496390,1066-09-09,other,en,,T10020_0.9312;T10622_0.0146;T10425_0.0054,Process (computing)_0.1987;Identification (bio...,0,T10020,106,1066-09-09
3550737,W7017208566,1088-01-01,article,es,,T13650_0.0409;T10885_0.0312;T12859_0.0227,Context (archaeology)_0.2093;Stage (stratigrap...,0,T13650,108,1088-01-01
...,...,...,...,...,...,...,...,...,...,...,...
4245919,W7077176216,2030-01-01,article,ceb,,T12157_0.6951;T13067_0.0271;T14311_0.0162,,0,T12157,203,2030-01-01
4245888,W7081571635,2040-09-16,other,es,10.5281/zenodo.17106754,T12157_0.6794;T13067_0.0295;T10271_0.0167,,0,T12157,204,2040-09-16
3238843,W6969175033,2044-04-13,article,en,10.5281/zenodo.9276,T13650_0.0272;T14406_0.0223;T13141_0.0206,Equilateral triangle_0.8595;Axiom_0.5168;Plane...,0,T13650,204,2044-04-13
0,W7026753882,2050-01-01,book,es,,T13398_0.1451;T10978_0.0384;T13937_0.0324,Order (exchange)_0.2391;Set (abstract data typ...,0,T13398,205,2050-01-01


In [41]:
top10_by_decade_filtered.columns

Index(['id', 'date', 'type', 'language', 'doi', 'topics', 'keywords',
       'cited_by_count_computed', 'primary_topic'],
      dtype='object')

In [40]:
top10_by_decade_filtered = top10_by_decade[(top10_by_decade.date >= '1950')&(top10_by_decade.date <= '2025')].drop(columns=['decade3', 'date_str'])
top10_by_decade_filtered

,id,date,type,language,doi,topics,keywords,cited_by_count_computed,primary_topic
2121190,W2040870580,1958-01-01,article,en,10.1037/h0042519,T10320_0.9989;T10581_0.9845;T12611_0.9552,Probabilistic logic_0.62432396;Artificial inte...,11046,T10320
3709079,W2001771035,1950-10-01,article,en,10.1093/mind/lix.236.433,T10215_0.0468;T12002_0.0451,Computer science_0.47242367;Cognitive science_...,8699,T10215
2556472,W2102236934,1958-01-01,article,en,10.2307/1907382,T10637_0.3068;T11303_0.2863;T11235_0.2767,Estimation_0.55080956;Econometrics_0.53709877;...,6713,T10637
1299054,W2014268383,1956-09-01,article,en,10.1214/aoms/1177728190,T11901_0.983;T10136_0.9598;T11871_0.9046,Mathematics_0.8505969;Univariate_0.67926306;Pr...,4347,T11901
3060314,W1483568252,1952-01-01,book,en,,T11574_0.0822,Philosophy_0.32179356,4174,T11574
3060200,W3198350258,1959-07-01,article,en,10.1147/rd.33.0210,T11574_0.8088,Computer science_0.71883404;Artificial intelli...,4099,T11574
2121328,W2158595960,1956-10-01,article,en,10.6028/jres.057.026,T10320_0.8117;T12564_0.7967;T12852_0.7847,Isothermal process_0.5557319;Analytical Chemis...,3768,T10320
7920518,W2064972007,1952-01-01,article,en,10.1130/0016-7606(1952)63[1117:haaoet]2.0.co;2,T12157_0.9969;T10770_0.9916;T14339_0.9695,Geology_0.778937;Altitude (triangle)_0.477409;...,3386,T12157
1851298,W1965612527,1958-06-01,article,en,10.1214/aoms/1177706645,T11975_0.214;T11574_0.2108,Mathematics_0.40054876,3251,T11975
2490751,W1974339500,1958-04-01,article,en,10.1147/rd.22.0159,T10181_0.573,IBM_0.73995435;Computer science_0.7201025;Natu...,3133,T10181


In [42]:
import os
import pandas as pd
from tqdm import tqdm

works2text_parquet_folder = "data/works2text_by_topic_parquet/"

# what you want to add
text_cols_wanted = ["title", "abstract"]

out_parts = []

for topic, part in tqdm(top10_by_decade_filtered.groupby("primary_topic", sort=False), total=top10_by_decade_filtered["primary_topic"].nunique()):
    text_path = os.path.join(works2text_parquet_folder, f"{topic}.parquet")

    # read only needed columns + id column (name might differ)
    text_df = pd.read_parquet(text_path)

    # --- figure out the id column in text_df ---
    # common possibilities: "work_id", "id"
    if "work_id" in text_df.columns:
        text_id_col = "work_id"
    elif "id" in text_df.columns:
        text_id_col = "id"
    else:
        raise KeyError(f"No work id column found in {text_path}. Columns: {list(text_df.columns)[:20]}")

    # keep only necessary columns that exist
    cols_to_keep = [text_id_col] + [c for c in text_cols_wanted if c in text_df.columns]
    text_df = text_df[cols_to_keep].drop_duplicates(subset=[text_id_col])

    # merge into this topic slice
    merged = part.merge(
        text_df,
        left_on="id",
        right_on=text_id_col,
        how="left"
    )

    # drop duplicate join key if needed
    if text_id_col != "id":
        merged = merged.drop(columns=[text_id_col])

    out_parts.append(merged)

works_df_with_text = pd.concat(out_parts, ignore_index=True)


100%|██████████| 29/29 [00:41<00:00,  1.44s/it]


In [43]:
works_df_with_text

,id,date,type,language,doi,topics,keywords,cited_by_count_computed,primary_topic,title,abstract
0,W2040870580,1958-01-01,article,en,10.1037/h0042519,T10320_0.9989;T10581_0.9845;T12611_0.9552,Probabilistic logic_0.62432396;Artificial inte...,11046,T10320,The perceptron: A probabilistic model for info...,
1,W2158595960,1956-10-01,article,en,10.6028/jres.057.026,T10320_0.8117;T12564_0.7967;T12852_0.7847,Isothermal process_0.5557319;Analytical Chemis...,3768,T10320,Variation of peak temperature with heating rat...,"In differe ntial thermal analys is, the temp e..."
2,W2178225550,1969-08-01,article,en,10.2307/1912791,T10320_0.2208;T12157_0.1979;T10876_0.1821,Econometrics_0.70911944;Economics_0.5986589;Ec...,21960,T10320,Investigating Causal Relations by Econometric ...,There occurs on some occasions a difficulty in...
3,W1784695092,1966-01-01,book,en,,T10320_0.3335;T12879_0.3013,Psychophysics_0.8971933;Detection theory_0.587...,9977,T10320,Signal detection theory and psychophysics,Book on statistical decision theory and sensor...
4,W2152825437,1964-03-01,article,en,10.1007/bf02289565,T10320_0.2956;T11063_0.2611;T10050_0.2376,Goodness of fit_0.8799572;Multidimensional sca...,7128,T10320,Multidimensional Scaling by Optimizing Goodnes...,Multidimensional scaling is the problem of rep...
5,W4301253858,1966-05-01,article,en,10.2307/1266379,T10320_0.0483;T11195_0.046;T11918_0.0415,Mathematics_0.50743693;Random variable_0.45491...,6266,T10320,"Probability, Random Variables, and Stochastic ...",
6,W1498436455,1986-10-01,article,en,10.1038/323533a0,T10320_0.9999;T11236_0.9908;T11447_0.9778,Computer science_0.729864;Task (project manage...,28763,T10320,Learning representations by back-propagating e...,
7,W3085162807,1986-03-01,article,en,10.2307/2288003,T10320_0.5464,Statistics_0.5579234;Regression_0.52488226;Mat...,20975,T10320,Classification and Regression Trees.,
8,W2137983211,1989-01-01,article,en,10.1016/0893-6080(89)90020-8,T10320_0.9987;T11522_0.9884;T11206_0.9824,Feed forward_0.8074792;Feedforward neural netw...,20151,T10320,Multilayer feedforward networks are universal ...,
9,W2064675550,1997-11-01,article,en,10.1162/neco.1997.9.8.1735,T10320_0.9716;T11307_0.9227,Term (time)_0.62981373;Short-term memory_0.488...,90801,T10320,Long Short-Term Memory,Learning to store information over extended ti...


In [44]:
import numpy as np

# OpenAlex work URL
works_df_with_text["openalex_url"] = "https://openalex.org/" + works_df_with_text["id"].astype("string")

# DOI URL (safe for missing/blank dois)
doi = works_df_with_text["doi"].astype("string").str.strip()
works_df_with_text["doi_url"] = np.where(
    doi.notna() & (doi != ""),
    "https://doi.org/" + doi,
    pd.NA
)
works_df_with_text

,id,date,type,language,doi,topics,keywords,cited_by_count_computed,primary_topic,title,abstract,openalex_url,doi_url
0,W2040870580,1958-01-01,article,en,10.1037/h0042519,T10320_0.9989;T10581_0.9845;T12611_0.9552,Probabilistic logic_0.62432396;Artificial inte...,11046,T10320,The perceptron: A probabilistic model for info...,,https://openalex.org/W2040870580,https://doi.org/10.1037/h0042519
1,W2158595960,1956-10-01,article,en,10.6028/jres.057.026,T10320_0.8117;T12564_0.7967;T12852_0.7847,Isothermal process_0.5557319;Analytical Chemis...,3768,T10320,Variation of peak temperature with heating rat...,"In differe ntial thermal analys is, the temp e...",https://openalex.org/W2158595960,https://doi.org/10.6028/jres.057.026
2,W2178225550,1969-08-01,article,en,10.2307/1912791,T10320_0.2208;T12157_0.1979;T10876_0.1821,Econometrics_0.70911944;Economics_0.5986589;Ec...,21960,T10320,Investigating Causal Relations by Econometric ...,There occurs on some occasions a difficulty in...,https://openalex.org/W2178225550,https://doi.org/10.2307/1912791
3,W1784695092,1966-01-01,book,en,,T10320_0.3335;T12879_0.3013,Psychophysics_0.8971933;Detection theory_0.587...,9977,T10320,Signal detection theory and psychophysics,Book on statistical decision theory and sensor...,https://openalex.org/W1784695092,<NA>
4,W2152825437,1964-03-01,article,en,10.1007/bf02289565,T10320_0.2956;T11063_0.2611;T10050_0.2376,Goodness of fit_0.8799572;Multidimensional sca...,7128,T10320,Multidimensional Scaling by Optimizing Goodnes...,Multidimensional scaling is the problem of rep...,https://openalex.org/W2152825437,https://doi.org/10.1007/bf02289565
5,W4301253858,1966-05-01,article,en,10.2307/1266379,T10320_0.0483;T11195_0.046;T11918_0.0415,Mathematics_0.50743693;Random variable_0.45491...,6266,T10320,"Probability, Random Variables, and Stochastic ...",,https://openalex.org/W4301253858,https://doi.org/10.2307/1266379
6,W1498436455,1986-10-01,article,en,10.1038/323533a0,T10320_0.9999;T11236_0.9908;T11447_0.9778,Computer science_0.729864;Task (project manage...,28763,T10320,Learning representations by back-propagating e...,,https://openalex.org/W1498436455,https://doi.org/10.1038/323533a0
7,W3085162807,1986-03-01,article,en,10.2307/2288003,T10320_0.5464,Statistics_0.5579234;Regression_0.52488226;Mat...,20975,T10320,Classification and Regression Trees.,,https://openalex.org/W3085162807,https://doi.org/10.2307/2288003
8,W2137983211,1989-01-01,article,en,10.1016/0893-6080(89)90020-8,T10320_0.9987;T11522_0.9884;T11206_0.9824,Feed forward_0.8074792;Feedforward neural netw...,20151,T10320,Multilayer feedforward networks are universal ...,,https://openalex.org/W2137983211,https://doi.org/10.1016/0893-6080(89)90020-8
9,W2064675550,1997-11-01,article,en,10.1162/neco.1997.9.8.1735,T10320_0.9716;T11307_0.9227,Term (time)_0.62981373;Short-term memory_0.488...,90801,T10320,Long Short-Term Memory,Learning to store information over extended ti...,https://openalex.org/W2064675550,https://doi.org/10.1162/neco.1997.9.8.1735


In [48]:
works_df_with_text.to_csv("data/ai_top_papers.csv", index = False, sep=',', header=True)

## 2) Helpers

In [7]:
def parse_year(date_str: str):
    if not date_str:
        return None
    try:
        return int(date_str[:4])
    except Exception:
        return None


def has_doi(doi: str) -> bool:
    return isinstance(doi, str) and len(doi.strip()) > 0


def has_title(title: str) -> bool:
    return isinstance(title, str) and len(title.strip()) > 0


def split_list_field(s: str) -> list[str]:
    if not isinstance(s, str) or len(s) == 0:
        return []
    return [x for x in s.split(";") if x]


def is_ai_broad(topics_str: str, ai_topics: set[str]) -> bool:
    # topics_str format: "T123_0.9;T456_0.8;..."
    if not isinstance(topics_str, str) or len(topics_str) == 0:
        return False
    for item in topics_str.split(";"):
        try:
            tid = item.split("_")[0]
        except Exception:
            continue
        if tid in ai_topics:
            return True
    return False


def is_ai_strict(primary_topic: str, ai_topics: set[str]) -> bool:
    return isinstance(primary_topic, str) and primary_topic in ai_topics


## 3) Load citation edges (optional)
We want **citations per work** (incoming). We build a map `{work_id -> [citing_work_ids]}` by grouping on `referenced_work_id`.

This can be heavy; in test mode, it scans only a few citation parquet files.


In [8]:
def load_topic_references_map(topic_id: str) -> dict[str, list[str]]:
    path = DATA / "works2references_by_topic_parquet" / f"{topic_id}.parquet"
    if not path.exists():
        return {}
    df = pl.read_parquet(path, columns=["work_id", "referenced_work_id"])
    grouped = df.group_by("work_id").agg(pl.col("referenced_work_id").implode())
    return {row["work_id"]: list(row["referenced_work_id"]) for row in grouped.iter_rows(named=True)}


def load_topic_citations_map(topic_id: str) -> dict[str, list[str]]:
    path = DATA / "works2citations_by_topic_parquet" / f"{topic_id}.parquet"
    if not path.exists():
        return {}
    df = pl.read_parquet(path, columns=["referenced_work_id", "work_id"])
    grouped = df.group_by("referenced_work_id").agg(pl.col("work_id").implode())
    return {row["referenced_work_id"]: list(row["work_id"]) for row in grouped.iter_rows(named=True)}

def load_topic_related_map(topic_id: str) -> dict[str, list[str]]:
    path = DATA / "works2related_by_topic_parquet" / f"{topic_id}.parquet"
    if not path.exists():
        return {}
    df = pl.read_parquet(path, columns=["work_id", "referenced_work_id"])
    grouped = df.group_by("work_id").agg(pl.col("referenced_work_id").implode())
    return {row["work_id"]: list(row["referenced_work_id"]) for row in grouped.iter_rows(named=True)}



## 4) Build AI works table
We scan topic-partitioned works parquet files, filter rows, and assemble the required columns.

Required columns in output:
- `work_id`, `date`, `doi`, `topics`, `keywords`, `authors`, `title`, `abstract`, `references`, `citations`, `primary_venue_id`

Note: in the processed works table, the **journal** column is the primary venue ID. We map it to `primary_venue_id`.


In [9]:
def iter_topic_files(works_dir: Path, test_mode: bool = False, max_topics: int = 3):
    files = sorted(works_dir.glob("*.parquet"))
    if test_mode:
        random.Random(RANDOM_SEED).shuffle(files)
        files = files[:max_topics]
    return files


def build_ai_table():
    rows = []
    files = iter_topic_files(WORKS_PARQUET_DIR, TEST_MODE, TEST_MAX_TOPICS)
    for f in tqdm(files):
        topic_id = f.stem

        references_map = load_topic_references_map(topic_id)
        citations_map = load_topic_citations_map(topic_id)

        # Base metadata from works_by_topic
        df = pl.read_parquet(f, columns=[
            "id", "date", "type", "language", "journal", "doi", "primary_topic", "topics", "keywords", "authors", "references", "related_works"
        ])

        # Join title/abstract from works2text_by_topic_parquet
        text_path = (DATA / "works2text_by_topic_parquet" / f"{topic_id}.parquet")
        if text_path.exists():
            text_df = pl.read_parquet(text_path, columns=["id", "title", "abstract"])
            df = df.join(text_df, on="id", how="left")
        else:
            df = df.with_columns([
                pl.lit("").alias("title"),
                pl.lit("").alias("abstract")
            ])

        # # Join topics from works2topic_by_topic_parquet (t0/s0/t1/s1/t2/s2)
        # topics_path = (DATA / "works2topic_by_topic_parquet" / f"{topic_id}.parquet")
        # if topics_path.exists():
        #     tdf = pl.read_parquet(topics_path, columns=["id", "t0", "s0", "t1", "s1", "t2", "s2"])
        
        #     tdf = tdf.with_columns([
        #         pl.when(pl.col("t0").is_not_null() & pl.col("s0").is_not_null())
        #           .then(pl.concat_str(pl.col("t0"), pl.col("s0"), separator="_"))
        #           .otherwise(None).alias("ts0"),
        #         pl.when(pl.col("t1").is_not_null() & pl.col("s1").is_not_null())
        #           .then(pl.concat_str(pl.col("t1"), pl.col("s1"), separator="_"))
        #           .otherwise(None).alias("ts1"),
        #         pl.when(pl.col("t2").is_not_null() & pl.col("s2").is_not_null())
        #           .then(pl.concat_str(pl.col("t2"), pl.col("s2"), separator="_"))
        #           .otherwise(None).alias("ts2"),
        #     ])
        
        #     tdf = tdf.with_columns(
        #         pl.concat_str(
        #             pl.col("ts0").fill_null(""),
        #             pl.col("ts1").fill_null(""),
        #             pl.col("ts2").fill_null(""),
        #             separator=";",
        #         )
        #         .str.replace_all(";;+", ";")
        #         .str.strip_chars(";")
        #         .alias("topics")
        #     ).select(["id", "topics"])
        
        #     df = df.join(tdf, on="id", how="left")
        # else:
        #     df = df.with_columns(pl.lit("").alias("topics"))

        # # Join authors from author2work_by_topic_parquet
        # authors_path = (DATA / "author2work_by_topic_parquet" / f"{topic_id}.parquet")
        # if authors_path.exists():
        #     adf = (
        #         pl.read_parquet(authors_path, columns=["work_id", "author_id"])
        #         .group_by("work_id")
        #         .agg(pl.col("author_id").implode().alias("authors"))
        #         .rename({"work_id": "id"})
        #     )
        #     df = df.join(adf, on="id", how="left")
        # else:
        #     df = df.with_columns(pl.lit("").alias("authors"))

        # # TODO: keywords from works2keywords folder (need path + columns)
        # # fallback: keep empty if not available
        # df = df.with_columns(pl.lit("").alias("keywords"))

        # basic filters
        df = df.filter(
            (pl.col("type").is_in(list(ALLOWED_TYPES))) &
            (pl.col("language") == LANGUAGE) &
            (pl.col("doi").is_not_null()) &
            (pl.col("title").is_not_null())
        )

        # date filter
        df = df.with_columns(pl.col("date").str.slice(0, 4).cast(pl.Int32).alias("year"))
        df = df.filter(pl.col("year") >= MIN_YEAR)

        # AI filter (broad)
        if AI_TOPIC_IDS:
            df = df.filter(
                pl.col("topics").map_elements(lambda s: is_ai_broad(s, AI_TOPIC_IDS), return_dtype=pl.Boolean)
            )
        else:
            print("WARNING: AI_TOPIC_IDS is empty; no AI filter applied.")

        for row in df.iter_rows(named=True):
            work_id = row["id"]
            references = row["references"]
            # references = ";".join(references_map.get(work_id, [])) if references_map else ""
            citations = ";".join(citations_map.get(work_id, [])) if citations_map else ""

            # must have >=1 reference or >=1 citation
            if len(references) == 0 and len(citations) == 0:
                continue

            # enforce non-empty title and DOI (safety)
            if not has_title(row.get("title")):
                continue
            if not has_doi(row.get("doi")):
                continue

            rows.append({
                "work_id": work_id,
                "date": row.get("date"),
                "doi": row.get("doi"),
                "topics": row.get("topics") or "",
                "title": row.get("title") or "",
                "abstract": row.get("abstract") or "",
                "keywords": row.get("keywords") or "",
                "authors": row.get("authors") or "",
                "references": references,
                "citations": citations,
                "related_works": row.get("related_works") or "",
                "primary_venue_id": row.get("journal") or "",
                "ai_strict": is_ai_strict(row.get("primary_topic"), AI_TOPIC_IDS) if AI_TOPIC_IDS else False,
                "ai_broad": True,
            })

    out = pl.DataFrame(rows)
    return out

ai_df = build_ai_table()
print("AI rows:", ai_df.height)
ai_df.head()


  0%|          | 0/4516 [00:00<?, ?it/s]

AI rows: 4651159


work_id,date,doi,topics,title,abstract,keywords,authors,references,citations,related_works,primary_venue_id,ai_strict,ai_broad
str,str,str,str,str,str,str,str,str,str,str,str,bool,bool
"""W7126392913""","""2026-01-31""","""10.1038/s41598-026-37782-7""","""T10001_0.4855;T10710_0.2927;T1…","""Geochemical cycling of arsenic…","""Arsenic (As) cycling in magmat…","""Supercontinent_0.8371;Igneous …","""A5124588166_I3125743391_T;A503…","""W415441523;W1498987157;W160205…","""""","""""","""S196734849""",false,true
"""W7126229298""","""2026-01-30""","""10.3390/min16020160""","""T10001_0.981;T12157_0.0045;T11…","""Origin of Archean Orogenic Gol…","""The Atlantic City–South Pass (…","""Fluid inclusions_0.806;Archean…","""A5124302654_I173911158|I130649…","""W46372954;W349591873;W10251434…","""""","""""","""S2737336974""",false,true
"""W7126228185""","""2026-01-30""","""10.3390/min16020162""","""T10001_0.4218;T11740_0.1884;T1…","""Supergene Alteration of Skarn …","""At the Ity gold deposit (Ivory…","""Saprolite_0.9549;Hypogene_0.86…","""A5044820503_I48615539_F;A51243…","""W958731027;W1723029166;W197282…","""""","""""","""S2737336974""",false,true
"""W7126188424""","""2026-01-30""","""10.3390/min16020164""","""T10001_0.2852;T11740_0.2506;T1…","""Genesis and Mineralization Pro…","""The Lanuoma sediment-hosted le…","""Sphalerite_0.9248;Sedimentary …","""A5050654783_I4210117772_T;A503…","""W630502682;W1970752532;W197176…","""""","""""","""S2737336974""",false,true
"""W7125714665""","""2026-01-26""","""10.1016/j.oregeorev.2026.10714…","""T10001_0.6668;T12157_0.1264;T1…","""Orogenic gold mineralization c…","""""","""Sphalerite_0.8594;Mineralizati…","""A5123837374_I3125743391_F;A511…","""W1025143415;W1748073762;W17972…","""""","""""","""S29724373""",false,true


In [10]:
ai_df = ai_df.sort(["date", "work_id"])

In [ ]:
ai_df_pd = pd.DataFrame(ai_df.to_dict())

In [ ]:
ai_df_pd.ai_strict.value_counts()

In [ ]:
ai_df_pd

In [1]:
print("C")

C


## 5) Save output

In [11]:
out_path = OUTPUT_DIR / "ai_works.parquet"
ai_df.write_parquet(out_path, compression="brotli")
print("Saved", out_path)


Saved /data/scratch/ahw701/openalex/OpenAlex-Dataset-Creation/data/ai_database/ai_works.parquet


In [9]:
out_path = OUTPUT_DIR / "ai_works.parquet"
ai_df = pl.read_parquet(out_path)
ai_df

work_id,date,doi,topics,title,abstract,keywords,authors,references,citations,related_works,primary_venue_id,ai_strict,ai_broad
str,str,str,str,str,str,str,str,str,str,str,str,bool,bool
"""W1442647268""","""1950-01-01""","""10.2467/mripapers1950.1.1_50""","""T10320_0.5995;T11918_0.5828;T1…","""On Stochastic Interpolation of…","""Most series of meteorological …","""Series (stratigraphy)_0.763776…","""A5018274244_I4388446413_T""","""W2032892544""","""W4313612012;W2153233078;W23392…","""W2167528397;W2511372218;W31957…","""S99792956""",true,true
"""W1483622850""","""1950-01-01""","""10.2307/4342693""","""T12353_0.8858;T10181_0.834;T13…","""Lemprière's Classical Dictiona…","""""","""Table (database)_0.75711036;Hi…","""A5112439191__T;A5046835724__F""","""""","""W2108152021;W2116616645;W87709…","""W2948670949;W4288047943;W42324…","""S4210182089""",false,true
"""W1491082791""","""1950-01-01""","""10.1049/sqj.1950.0079""","""T12838_0.2551;T13514_0.2439;T1…","""The Theory and Design of Elect…","""by J. R. Pierce""","""Electron_0.5045978;Designtheor…","""A5033440148__T""","""""","""W1965256673;W2169755275;W21406…","""W2078200235;W2082591874;W28899…","""S4210189634""",false,true
"""W1492739685""","""1950-01-01""","""10.3133/b969a""","""T10572_0.9577;T14339_0.9342;T1…","""Diamond-drill exploration of t…","""""","""Magnetite_0.7218568;Drill_0.70…","""A5056633373__T""","""""","""W2154583868;W411349560;W252965…","""W989007996;W2372515719;W196488…","""""",false,true
"""W1504310735""","""1950-01-01""","""10.5928/kaiyou1942.6.68""","""T13650_0.8643;T11405_0.7998;T1…","""On the Neglect of the Inertia …","""""","""Neglect_0.75007355;Inertia_0.6…","""A5040871555__T;A5003224324__F""","""""","""W1971625182""","""W2159755425;W1969705708;W23351…","""S2764687577""",true,true
"""W1512469077""","""1950-01-01""","""10.3133/mf1""","""T12773_0.9208;T12157_0.915""","""Geologic map of the central pa…","""""","""Geology_0.75576127;Independenc…","""A5071931393__T;A5008381984__F""","""""","""W2947361727""","""W2349657250;W4210628558;W42058…","""""",false,true
"""W1512742907""","""1950-01-01""","""10.3133/b955d""","""T12157_0.8574;T13065_0.852;T11…","""Gold placers and their geologi…","""The placer district of northwe…","""Archaeology_0.61410904;Geology…","""A5100097161__T""","""W6601822323;W6607397737""","""W2034777801;W4253074623;W19890…","""W2899084033;W2748952813;W23246…","""""",true,true
"""W1522252020""","""1950-01-01""","""10.1016/s0016-7878(50)80024-x""","""T10535_0.894;T13650_0.8607;T12…","""Gravitational sliding""","""""","""Physics_0.4555598;Classical me…","""A5024746828__T""","""W1506956272;W1558638743;W20127…","""W4282913291;W2158242164""","""W2039255853;W2086733001;W21671…","""S4210171952""",false,true
"""W1528543422""","""1950-01-01""","""10.3133/ofr5033""","""T12157_0.8196;T11089_0.8002;T1…","""An improved fluorimeter for th…","""Abstract: The Model R fluorime…","""Fluorometer_0.9899718;Sensitiv…","""A5017540566__T;A5109007545__F""","""""","""W1597537097;W2513885859;W14833…","""W4387497383;W2948807893;W27781…","""S4210194219""",true,true


In [9]:
ai_df.columns

['work_id',
 'date',
 'doi',
 'topics',
 'title',
 'abstract',
 'keywords',
 'authors',
 'references',
 'citations',
 'related_works',
 'primary_venue_id',
 'ai_strict',
 'ai_broad']

In [8]:
ai_df.select(pl.col("ai_strict").value_counts())

ai_strict
struct[2]
"{false,2304388}"
"{true,2346771}"


## 6) Simplicial Complex from Keywords (Weighted)
We treat **keywords** as nodes. For each paper with `n` keywords, we add all simplices of order `k` (size `k`) with weight `1 / C(n, k)` so the total weight contributed by a paper is 1 across all orders (Newman collaboration-style normalization).

This is a starting point ? optimize for scale later.


In [ ]:
# import itertools
# import math
# from collections import defaultdict


# def comb(n, k):
#     return math.comb(n, k)


# def keyword_simplicial_weights(keyword_list: list[str], max_k: int | None = None):
#     # Return dict: simplex (tuple) -> weight
#     kws = [k for k in keyword_list if k]
#     if len(kws) == 0:
#         return {}
#     if max_k is None:
#         max_k = len(kws)
#     weights = defaultdict(float)
#     n = len(kws)
#     for k in range(1, min(max_k, n) + 1):
#         denom = comb(n, k)
#         if denom == 0:
#             continue
#         w = 1.0 / denom
#         for simplex in itertools.combinations(sorted(set(kws)), k):
#             weights[simplex] += w
#     return weights

# # Example: build weights for a small sample
# example = ai_df.head(50)
# S = defaultdict(float)
# for row in example.iter_rows(named=True):
#     kws = split_list_field(row["keywords"])
#     w = keyword_simplicial_weights(kws)
#     for s, val in w.items():
#         S[s] += val

# print("#simplices:", len(S))


## 7) Filtration on S and Betti curves (placeholder)
We will build a filtration over intrinsic time (paper order) and real time (date). Add your preferred TDA library here (e.g., `gudhi`, `ripser`, or custom).

In [ ]:
# TODO: add Betti curve computation using your preferred library


## 8) Author-based example (placeholder)
Filter to a specific author (or group) and rebuild the simplicial complex over time.

In [ ]:
# TODO: implement author-based subgraph filtering


## 9) Novelty counts per paper (placeholder)
Track newly introduced simplices by order and distinguish novel higher-order simplices enabled by lower-order novelty.

In [ ]:
# TODO: implement novelty counting with order-aware tracking
